In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-05-08 11:51:35.987500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 11:51:36.058912: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 11:51:36.393070: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
import sys
sys.path.append('/home/carla-simulator/PythonAPI/carla') 
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [3]:
SECONDS_PER_EPISODE = 20

N_CHANNELS = 3
HEIGHT = 250
WIDTH = 250

FIXED_DELTA_SECONDS = 0.2

SHOW_PREVIEW = True

In [4]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    PREFERRED_SPEED = 60
    SPEED_THRESHOLD = 2
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.action_space = spaces.Discrete(9)

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
        
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)
        self.client.load_world('Town01')
        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        #self.settings.no_rendering_mode = True
        #self.settings.synchronous_mode = False
        #self.settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
        self.world.apply_settings(self.settings)
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter('*mini*')[0]
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        self.start_point = self.spawn_points[2]

        # Define point A and point B
        self.point_a = self.start_point
        self.point_b = carla.Location(x=107.175446, y=129.416809, z=0.003263)  # Destination point

        self.define_waypoints()
        
    def maintain_speed(self, s):
        if s >= self.PREFERRED_SPEED:
            return 0
        elif s < self.PREFERRED_SPEED - self.SPEED_THRESHOLD:
            return 0.7 # think of it as % of "full gas"
        else:
            return 0.3 # tweak this if the car is way over or under preferred speed 
        


    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        cv2.destroyAllWindows()

    def reset(self, seed=1, options=None):
        super().reset(seed=seed)
        self.cleanup()

        self.collision_hist = []
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        self.start_point = self.spawn_points[2]

        # Define point A and point B
        self.point_a = self.start_point
        self.point_b = carla.Location(x=107.175446, y=129.416809, z=0.003263)  # Destination point

        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.start_point)
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()  # Clean up any existing actors
            return self.reset()  # Retry resetting the environment

        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        self.initial_location = self.vehicle.get_location()

        # Define waypoints between point A and point B
        self.define_waypoints()

        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")

        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z,x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2) #not detect a collision when the car spawns/falls from sky.
        
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))

        while self.front_camera is None:
            time.sleep(0.01)#Just in case car takes any longer, because we need to be certain the car is done falling from the sky on spawn.

        self.episode_start = time.time()

        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))

        observation = self.front_camera

        return observation.astype(np.uint8), {}

    
    def compute_waypoints(self, start, end, num_waypoints=10):
        """
        Compute waypoints between start and end positions.

        Args:
            start (carla.Location): Starting position.
            end (carla.Location): Ending position.
            num_waypoints (int): Number of waypoints to compute.

        Returns:
            List of carla.Location: Waypoints between start and end positions.
        """
        # Compute the direction vector from start to end
        direction = np.array([end.x - start.x, end.y - start.y])
        distance = np.linalg.norm(direction)
        unit_direction = direction / distance

        # Compute the distance between waypoints
        step_size = distance / num_waypoints

        # Compute waypoints
        waypoints = []
        for i in range(num_waypoints):
            waypoint_x = start.x + unit_direction[0] * step_size * i
            waypoint_y = start.y + unit_direction[1] * step_size * i
            waypoint_z = start.z  # Assuming same z-coordinate for waypoints
            waypoint = carla.Location(x=waypoint_x, y=waypoint_y, z=waypoint_z)
            waypoints.append(waypoint)

        return waypoints

    
    def define_waypoints(self):
        # Define start and end points
        start_point = self.start_point.location  # Use start_point attribute instead of point_a
        end_point = carla.Location(x=107.175446, y=129.416809, z=0.003263)  # Destination point

        # Compute waypoints between start and end positions
        self.waypoints = self.compute_waypoints(start_point, end_point, num_waypoints=50)
        # Draw waypoints on the map
        for waypoint in self.waypoints:
            self.world.debug.draw_string(waypoint, '^', draw_shadow=False,
                                          color=carla.Color(r=0, g=0, b=255), life_time=600.0,
                                          persistent_lines=True)


    
    def step(self, action):
        self.step_counter += 1
        steer = action

        # Follow waypoints instead of direct control
        next_waypoint = self.waypoints[action]
        target_location = next_waypoint
        vehicle_location = self.vehicle.get_location()
        # Calculate direction vector from vehicle to target
        direction = np.array([target_location.x - vehicle_location.x, target_location.y - vehicle_location.y])
        # Calculate target rotation angle
        target_rotation = np.arctan2(direction[1], direction[0]) * 180 / np.pi
        # Get current vehicle rotation
        current_rotation = self.vehicle.get_transform().rotation.yaw
        # Calculate the difference between target and current rotation
        angle_difference = target_rotation - current_rotation
        # Adjust the steering based on the angle difference
        #angle_difference is always within the range [-180, 180]
        if angle_difference > 180:
            angle_difference -= 360
        elif angle_difference < -180:
            angle_difference += 360
            
        # Normalize the angle difference to be in range [-180, 180]
        steer = np.clip(angle_difference / 180, -1, 1)
       
 
        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        estimated_throttle = self.maintain_speed(kmh)

        self.vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer, brake=0.0))

        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        distance = self.point_a.location.distance(self.point_b)

        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera


        # start defining reward from each step
        reward = 0
        done = False
        #punish for collision
        if len(self.collision_hist) != 0:
            done = True
            reward = reward - 50
            #self.cleanup()
            # Reset the environment immediately after collision
            observation = self.reset()[0]
            
        # Calculate reward based on distance traveled towards point B    
        if distance_to_destination < distance/4:  # threshold for reaching destination
            reward = reward + 30  # Positive reward for reaching destination
            done = True  # End episode when destination is reached
        elif distance_to_destination < distance/3:
            reward = reward + 20  
        elif distance_to_destination < distance/2:
            reward = reward + 10  
        elif distance_to_destination < distance:
            reward = reward + 5
        
        else:
            reward = reward - 10  # Negative reward for not reaching destination
            done = False

        # check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
            self.cleanup()
            #print(self.actor_list)
            
        observation = cam

        return cam.astype(np.uint8), reward, done, False, {}

    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3] # this is to ignore the 4th Alpha channel - up to 3
        self.front_camera = i/255.0

    def collision_data(self, event):
        self.collision_hist.append(event)
 

In [5]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1,learning_rate=0.001,tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...
Trying to spawn vehicle...
Vehicle spawned successfully!
Logging to logs/1715140329/PPO_0
-----------------------------
| time/              |      |
|    fps             | 354  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------


/home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05e+03    |
|    ep_rew_mean          | 1.02e+04    |
| time/                   |             |
|    fps                  | 206         |
|    iterations           | 2           |
|    time_elapsed         | 19          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.003883122 |
|    clip_fraction        | 0.0125      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.2        |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 3.47e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00313    |
|    value_loss           | 6.95e+03    |
-----------------------------------------
Tryi

Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.05e+03      |
|    ep_rew_mean          | 1.02e+04      |
| time/                   |               |
|    fps                  | 155           |
|    iterations           | 10            |
|    time_elapsed         | 131           |
|    total_timesteps      | 20480         |
| train/                  |               |
|    approx_kl            | 0.00080440263 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.19         |
|    explained_variance   | 1.19e-07      |
|    learning_rate        | 0.001         |
|    loss                 | 3.38e+03      |
|    n_updates            | 90            |
|    policy_gradient_loss | -0.000657     |
|    value_loss           | 6.88e+03      |
-------------------------------------------
Trying to spawn veh

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.05e+03     |
|    ep_rew_mean          | 1.02e+04     |
| time/                   |              |
|    fps                  | 151          |
|    iterations           | 18           |
|    time_elapsed         | 243          |
|    total_timesteps      | 36864        |
| train/                  |              |
|    approx_kl            | 0.0021321597 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.19        |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 3.43e+03     |
|    n_updates            | 170          |
|    policy_gradient_loss | -0.00237     |
|    value_loss           | 6.81e+03     |
------------------------------------------
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawn

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.04e+03     |
|    ep_rew_mean          | 1.02e+04     |
| time/                   |              |
|    fps                  | 145          |
|    iterations           | 205          |
|    time_elapsed         | 2894         |
|    total_timesteps      | 1925120      |
| train/                  |              |
|    approx_kl            | 0.0009927718 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.92        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.001        |
|    loss                 | 547          |
|    n_updates            | 9390         |
|    policy_gradient_loss | -0.000922    |
|    value_loss           | 1.15e+03     |
------------------------------------------
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawn

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.05e+03     |
|    ep_rew_mean          | 1.02e+04     |
| time/                   |              |
|    fps                  | 145          |
|    iterations           | 214          |
|    time_elapsed         | 3020         |
|    total_timesteps      | 1943552      |
| train/                  |              |
|    approx_kl            | 0.0010219804 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.93        |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.001        |
|    loss                 | 523          |
|    n_updates            | 9480         |
|    policy_gradient_loss | -0.000437    |
|    value_loss           | 1.12e+03     |
--------------------------

Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.05e+03      |
|    ep_rew_mean          | 1.02e+04      |
| time/                   |               |
|    fps                  | 145           |
|    iterations           | 222           |
|    time_elapsed         | 3134          |
|    total_timesteps      | 1959936       |
| train/                  |               |
|    approx_kl            | 0.00058371783 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 508           |
|    n_updates            | 9560          |
|    policy_gradient_loss | -0.000582     |
|    value_loss           | 1.09e+03      |
-------------------------------------------
Trying to spawn veh

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.05e+03     |
|    ep_rew_mean          | 1.02e+04     |
| time/                   |              |
|    fps                  | 145          |
|    iterations           | 230          |
|    time_elapsed         | 3248         |
|    total_timesteps      | 1976320      |
| train/                  |              |
|    approx_kl            | 0.0016131841 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.96        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.001        |
|    loss                 | 708          |
|    n_updates            | 9640         |
|    policy_gradient_loss | -0.00179     |
|    value_loss           | 1.43e+03     |
--------------------------

Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.05e+03     |
|    ep_rew_mean          | 1.03e+04     |
| time/                   |              |
|    fps                  | 144          |
|    iterations           | 238          |
|    time_elapsed         | 3362         |
|    total_timesteps      | 1992704      |
| train/                  |              |
|    approx_kl            | 0.0031762589 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -2           |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.001        |
|    loss                 | 694          |
|    n_updates            | 9720         |
|    policy_gradient_loss | -0.00276     |
|    value_loss           | 1.42e+03     |
------------------------------------------
Trying to spawn vehicle...
Vehicle spawn

In [5]:
env = CarEnv()
env.reset()

models_dir = "/home/gp-coopperc/CARLA_TEST/models/1715140329/"
model_path = f"{models_dir}/2000000.zip"
model = PPO.load(model_path, env=env)

Trying to spawn vehicle...
Vehicle spawned successfully!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [6]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

env = Monitor(env) 
evaluate_policy(model, env, n_eval_episodes=5,deterministic=True, return_episode_rewards= True)

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!


([24055.0, 27550.0, 29410.0, 28685.0, 28645.0], [4821, 5520, 5892, 5747, 5739])